In [49]:
!pip install wrds

In [1]:
import wrds

In [3]:
db = wrds.Connection()

Enter your WRDS username [JAMHouse]:jose84
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
db.list_libraries()

['aha',
 'aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_corp_legal',
 'auditsmp',
 'bank',
 'blab',
 'block',
 'block_all_new',
 'boardex',
 'boardex_eur',
 'boardex_na',
 'boardex_row',
 'boardex_trial',
 'boardex_uk',
 'boardsmp',
 'bvd',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'centris',
 'ciq',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'cisdm',
 'cisdmsmp',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compdcur',
 'compg',
 'compgd',
 'comph',
 'compm',
 'compmcur',
 'compnad',
 'compsamp',
 'compsamp_snapshot',
 'compseg',
 'compsegd',
 'compsnap',
 'comscore',
 'contrib',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_general',
 'contrib_general_new',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_indexes',
 'crsp_a_stock',
 'crsp_a_sto

In [4]:
db.describe_table(library='crsp', table='msi')

Approximately 1141 rows in crsp.msi.


,name,nullable,type
0,date,True,DATE
1,vwretd,True,DOUBLE PRECISION
2,vwretx,True,DOUBLE PRECISION
3,ewretd,True,DOUBLE PRECISION
4,ewretx,True,DOUBLE PRECISION
5,sprtrn,True,DOUBLE PRECISION
6,spindx,True,DOUBLE PRECISION
7,totval,True,DOUBLE PRECISION
8,totcnt,True,DOUBLE PRECISION
9,usdval,True,DOUBLE PRECISION


In [5]:
db.describe_table(library='ff', table='factors_monthly')

Approximately 1136 rows in ff.factors_monthly.


,name,nullable,type
0,date,True,DATE
1,mktrf,True,DOUBLE PRECISION
2,smb,True,DOUBLE PRECISION
3,hml,True,DOUBLE PRECISION
4,rf,True,DOUBLE PRECISION
5,year,True,DOUBLE PRECISION
6,month,True,DOUBLE PRECISION
7,umd,True,DOUBLE PRECISION
8,dateff,True,DATE


In [4]:
import pandas as pd
import numpy as np

### 1. Dividend Yield의 Market Excess Return 예측 검증

In [7]:
rf = db.get_table(library='ff', table='factors_monthly')[['date','rf']]
rf['date'] = rf['date'] - pd.tseries.offsets.MonthEnd()

In [8]:
rf.head()

,date,rf
0,1926-06-30,0.0022
1,1926-07-31,0.0025
2,1926-08-31,0.0023
3,1926-09-30,0.0032
4,1926-10-31,0.0031


In [9]:
mkt = db.get_table(library='crsp', table='msi')[['date', 'vwretd', 'vwretx']]
mkt['date'] = mkt['date']-pd.tseries.offsets.MonthEnd()

In [10]:
mkt.head()

,date,vwretd,vwretx
0,1925-11-30,NaN,NaN
1,1925-12-31,0.000561,-0.001395
2,1926-01-31,-0.033046,-0.036587
3,1926-02-28,-0.064002,-0.070021
4,1926-03-31,0.037029,0.034043


In [11]:
df=pd.merge(mkt, rf, on='date')
df.head()

,date,vwretd,vwretx,rf
0,1926-06-30,0.031411,0.029224,0.0022
1,1926-07-31,0.028647,0.025358,0.0025
2,1926-08-31,0.005787,0.000030,0.0023
3,1926-09-30,-0.028996,-0.031069,0.0032
4,1926-10-31,0.028554,0.022979,0.0031


In [12]:
df['DP']=df['vwretd']-df['vwretx']
df['DP_lag']=df['DP'].shift(1)

In [13]:
df['exret']=df['vwretd']-df['rf']
df.head()

,date,vwretd,vwretx,rf,DP,DP_lag,exret
0,1926-06-30,0.031411,0.029224,0.0022,0.002186,NaN,0.029211
1,1926-07-31,0.028647,0.025358,0.0025,0.003290,0.002186,0.026147
2,1926-08-31,0.005787,0.000030,0.0023,0.005757,0.003290,0.003487
3,1926-09-30,-0.028996,-0.031069,0.0032,0.002073,0.005757,-0.032196
4,1926-10-31,0.028554,0.022979,0.0031,0.005575,0.002073,0.025454


In [14]:
df.date=df.date.astype(str)
df[['year', 'month', 'day']] = df.date.str.split("-", expand=True)
df.year = df.year.astype(int)
df=df[df.year<2021]

In [15]:
regression_data = df[['date', 'DP_lag', 'exret']]

In [16]:
regression_data=regression_data.dropna()
regression_data.head()

,date,DP_lag,exret
1,1926-07-31,0.002186,0.026147
2,1926-08-31,0.003290,0.003487
3,1926-09-30,0.005757,-0.032196
4,1926-10-31,0.002073,0.025454
5,1926-11-30,0.005575,0.026411


In [5]:
import statsmodels.formula.api as sm

In [21]:
result = sm.ols(formula="exret ~ DP_lag", data=regression_data).fit()

In [22]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  exret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.764
Date:                Sun, 25 Apr 2021   Prob (F-statistic):             0.0967
Time:                        20:36:12   Log-Likelihood:                 1710.0
No. Observations:                1133   AIC:                            -3416.
Df Residuals:                    1131   BIC:                            -3406.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.003      1.004      0.3

In [23]:
reg = sm.ols(formula="exret ~ DP_lag", data=regression_data).fit()
new = reg.get_robustcov_results(cov_type='HAC', maxlags=12)
print(new.summary())

                            OLS Regression Results                            
Dep. Variable:                  exret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.512
Date:                Sun, 25 Apr 2021   Prob (F-statistic):              0.113
Time:                        20:36:14   Log-Likelihood:                 1710.0
No. Observations:                1133   AIC:                            -3416.
Df Residuals:                    1131   BIC:                            -3406.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.003      1.103      0.2

In [24]:
import os
regression_data.to_csv('regression_divy.csv')

### 2. SMB, HML의 Market Excess Return 예측 검증

In [25]:
rf = db.get_table(library='ff', table='factors_monthly')[['date','rf']]
rf['date'] = rf['date'] - pd.tseries.offsets.MonthEnd()

In [50]:
rf.head()

,date,rf
0,1926-06-30,0.0022
1,1926-07-31,0.0025
2,1926-08-31,0.0023
3,1926-09-30,0.0032
4,1926-10-31,0.0031


In [27]:
mkt = db.get_table(library='crsp', table='msi')[['date', 'vwretd']]
mkt['date'] = mkt['date']-pd.tseries.offsets.MonthEnd()

In [28]:
mkt.head()

,date,vwretd
0,1925-11-30,NaN
1,1925-12-31,0.000561
2,1926-01-31,-0.033046
3,1926-02-28,-0.064002
4,1926-03-31,0.037029


In [31]:
smb_hml_df = db.get_table(library='ff', table='factors_monthly')[['date','smb', 'hml']]
smb_hml_df['date'] = smb_hml_df['date'] - pd.tseries.offsets.MonthEnd()

In [32]:
smb_hml_df.head()

,date,smb,hml
0,1926-06-30,-0.0230,-0.0287
1,1926-07-31,-0.0140,0.0419
2,1926-08-31,-0.0132,0.0001
3,1926-09-30,0.0004,0.0051
4,1926-10-31,-0.0020,-0.0035


In [33]:
exret_df=pd.merge(mkt, rf, on='date')
fama_df=pd.merge(exret_df, smb_hml_df, on='date')
fama_df.head()

,date,vwretd,rf,smb,hml
0,1926-06-30,0.031411,0.0022,-0.0230,-0.0287
1,1926-07-31,0.028647,0.0025,-0.0140,0.0419
2,1926-08-31,0.005787,0.0023,-0.0132,0.0001
3,1926-09-30,-0.028996,0.0032,0.0004,0.0051
4,1926-10-31,0.028554,0.0031,-0.0020,-0.0035


In [34]:
fama_df['exret']=fama_df['vwretd']-fama_df['rf']
fama_df.head()

,date,vwretd,rf,smb,hml,exret
0,1926-06-30,0.031411,0.0022,-0.0230,-0.0287,0.029211
1,1926-07-31,0.028647,0.0025,-0.0140,0.0419,0.026147
2,1926-08-31,0.005787,0.0023,-0.0132,0.0001,0.003487
3,1926-09-30,-0.028996,0.0032,0.0004,0.0051,-0.032196
4,1926-10-31,0.028554,0.0031,-0.0020,-0.0035,0.025454


In [35]:
fama_df.date=fama_df.date.astype(str)
fama_df[['year', 'month', 'day']] = fama_df.date.str.split("-", expand=True)
fama_df.year = fama_df.year.astype(int)
fama_df=fama_df[fama_df.year<2021]

In [36]:
regression_data = fama_df[['date','smb', 'hml','exret']]

In [37]:
regression_data=regression_data.dropna()
regression_data.head()

,date,smb,hml,exret
0,1926-06-30,-0.0230,-0.0287,0.029211
1,1926-07-31,-0.0140,0.0419,0.026147
2,1926-08-31,-0.0132,0.0001,0.003487
3,1926-09-30,0.0004,0.0051,-0.032196
4,1926-10-31,-0.0020,-0.0035,0.025454


In [39]:
result = sm.ols(formula="exret ~ smb + hml", data=regression_data).fit()

In [40]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  exret   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     97.26
Date:                Sun, 25 Apr 2021   Prob (F-statistic):           1.06e-39
Time:                        20:39:54   Log-Likelihood:                 1800.5
No. Observations:                1134   AIC:                            -3595.
Df Residuals:                    1131   BIC:                            -3580.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0047      0.001      3.145      0.0

In [41]:
reg = sm.ols(formula="exret ~ smb + hml", data=regression_data).fit()
new = reg.get_robustcov_results(cov_type='HAC', maxlags=12)
print(new.summary())

                            OLS Regression Results                            
Dep. Variable:                  exret   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     43.15
Date:                Sun, 25 Apr 2021   Prob (F-statistic):           8.73e-19
Time:                        20:39:57   Log-Likelihood:                 1800.5
No. Observations:                1134   AIC:                            -3595.
Df Residuals:                    1131   BIC:                            -3580.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0047      0.002      2.744      0.0

In [42]:
regression_data.to_csv('regression_sml_hml.csv')

### 3. T-Bill Rate의 Market Excess Return 예측 검증

In [6]:
db.list_tables(library='crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [7]:
db.describe_table(library='crsp', table='tfz_mth_rf')

Approximately 2282 rows in crsp.tfz_mth_rf.


,name,nullable,type
0,kytreasnox,True,DOUBLE PRECISION
1,mcaldt,True,DATE
2,rmtreasno,True,DOUBLE PRECISION
3,rmcrspid,True,VARCHAR(15)
4,tmbidytm,True,DOUBLE PRECISION
5,tmaskytm,True,DOUBLE PRECISION
6,tmytm,True,DOUBLE PRECISION
7,tmduratn,True,DOUBLE PRECISION


In [94]:
tb = db.get_table(library='crsp', table='tfz_mth_rf')[['mcaldt','tmytm']]
tb.columns = ["date", "tmytm"]

In [95]:
tb.head()

,date,tmytm
0,1925-12-31,3.637699
1,1926-01-30,3.673502
2,1926-02-27,3.134354
3,1926-03-31,3.121773
4,1926-04-30,2.724158


In [96]:
rf = db.get_table(library='ff', table='factors_monthly')[['date','rf']]
rf['date'] = rf['date'] - pd.tseries.offsets.MonthEnd()

In [97]:
rf.head()

,date,rf
0,1926-06-30,0.0022
1,1926-07-31,0.0025
2,1926-08-31,0.0023
3,1926-09-30,0.0032
4,1926-10-31,0.0031


In [98]:
mkt = db.get_table(library='crsp', table='msi')[['date', 'vwretd']]
mkt['date'] = mkt['date']-pd.tseries.offsets.MonthEnd()

In [99]:
mkt.head()

,date,vwretd
0,1925-11-30,NaN
1,1925-12-31,0.000561
2,1926-01-31,-0.033046
3,1926-02-28,-0.064002
4,1926-03-31,0.037029


In [100]:
exret_df=pd.merge(mkt, rf, on='date')
exret_df.head()

,date,vwretd,rf
0,1926-06-30,0.031411,0.0022
1,1926-07-31,0.028647,0.0025
2,1926-08-31,0.005787,0.0023
3,1926-09-30,-0.028996,0.0032
4,1926-10-31,0.028554,0.0031


In [101]:
exret_df.date=exret_df.date.astype(str)
exret_df[['year', 'month', 'day']] = exret_df.date.str.split("-", expand=True)
exret_df.year = exret_df.year.astype(int)
exret_df=exret_df[exret_df.year<2021]
exret_df.year = exret_df.year.astype(str)
exret_df['date'] = exret_df['year']+'-'+exret_df['month']
exret_df = exret_df.drop(['year','month','day'],1)

In [102]:
exret_df.head(20)

,date,vwretd,rf
0,1926-06,0.031411,0.0022
1,1926-07,0.028647,0.0025
2,1926-08,0.005787,0.0023
3,1926-09,-0.028996,0.0032
4,1926-10,0.028554,0.0031
5,1926-11,0.029211,0.0028
6,1926-12,0.002416,0.0025
7,1927-01,0.045370,0.0026
8,1927-02,0.003756,0.0030
9,1927-03,0.007225,0.0025


In [103]:
tb.date=tb.date.astype(str)
tb[['year', 'month', 'day']] = tb.date.str.split("-", expand=True)
tb.year = tb.year.astype(int)
tb=tb[tb.year<2021]
tb.year = tb.year.astype(str)
tb['date'] = tb['year']+'-'+tb['month']
tb = tb.drop(['year','month','day'],1)
tb = tb.drop_duplicates(['date'])

In [104]:
tb_df=pd.merge(exret_df, tb, on='date', how='inner')
tb_df.head()

,date,vwretd,rf,tmytm
0,1926-06,0.031411,0.0022,2.976560
1,1926-07,0.028647,0.0025,3.204387
2,1926-08,0.005787,0.0023,3.423826
3,1926-09,-0.028996,0.0032,3.685026
4,1926-10,0.028554,0.0031,3.557187


In [105]:
tb_df['exret']=tb_df['vwretd']-tb_df['rf']
tb_df.head()

,date,vwretd,rf,tmytm,exret
0,1926-06,0.031411,0.0022,2.976560,0.029211
1,1926-07,0.028647,0.0025,3.204387,0.026147
2,1926-08,0.005787,0.0023,3.423826,0.003487
3,1926-09,-0.028996,0.0032,3.685026,-0.032196
4,1926-10,0.028554,0.0031,3.557187,0.025454


In [106]:
regression_data = tb_df[['date', 'tmytm', 'exret']]

In [107]:
regression_data=regression_data.dropna()
regression_data.head()

,date,tmytm,exret
0,1926-06,2.976560,0.029211
1,1926-07,3.204387,0.026147
2,1926-08,3.423826,0.003487
3,1926-09,3.685026,-0.032196
4,1926-10,3.557187,0.025454


In [108]:
result = sm.ols(formula="exret ~ tmytm", data=regression_data).fit()

In [109]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  exret   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     5.694
Date:                Sun, 25 Apr 2021   Prob (F-statistic):             0.0172
Time:                        23:09:09   Log-Likelihood:                 1713.4
No. Observations:                1134   AIC:                            -3423.
Df Residuals:                    1132   BIC:                            -3413.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0107      0.002      4.609      0.0

In [110]:
reg = sm.ols(formula="exret ~ tmytm", data=regression_data).fit()
new = reg.get_robustcov_results(cov_type='HAC', maxlags=12)
print(new.summary())

                            OLS Regression Results                            
Dep. Variable:                  exret   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     5.923
Date:                Sun, 25 Apr 2021   Prob (F-statistic):             0.0151
Time:                        23:09:30   Log-Likelihood:                 1713.4
No. Observations:                1134   AIC:                            -3423.
Df Residuals:                    1132   BIC:                            -3413.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0107      0.003      4.025      0.0

In [111]:
regression_data.to_csv('regression_tbill.csv')